Import libraries:

In [1]:
from __future__ import unicode_literals
from nltk.tokenize import word_tokenize
import wikipediaapi
import string
    
import os
from tqdm import tqdm
from docx import Document
from docx.shared import Cm
from docx.shared import Pt
import concurrent.futures
from flask import Flask, render_template, request
from docx.enum.dml import MSO_THEME_COLOR_INDEX
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt
import docx
import time

from reportlab.lib.styles import ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER
from reportlab.platypus import Table, TableStyle
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.platypus import Image
from reportlab import platypus

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

import numpy as np
import pandas as pd
from tika import parser
from gensim.summarization.summarizer import summarize

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from requests import get
import random
import html
import re

import nltk
import collections
nltk.download('punkt')
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4.element import Comment

import networkx as nx
from nltk.tokenize import sent_tokenize

import spacy
import neuralcoref
from spacy.symbols import nsubj, nsubjpass, VERB

from selenium.webdriver.common.keys import Keys
import chromedriver_binary

from selenium import webdriver
from bs4 import BeautifulSoup
from bs4.element import Tag
from random import randint
import winsound

nlp = spacy.load('en_core_web_lg')
nlp.max_length = 50000000

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\skamenshchikov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Patameters:

In [2]:
UPLOAD_FOLDER = 'docs/'
ALLOWED_EXTENSIONS = set(['pdf', 'docx'])

readtime = 2
page_number = 5
keys_number = 10

process_time = 0

wiki_FT = True
google_FT = True
arxiv_FT = True

Import CDFs:

In [3]:
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

def get_unique_text(document):
    unique_sentences = []
    for sentence in [sent.raw for sent in TextBlob(document).sentences]:
        if sentence not in unique_sentences:
            unique_sentences.append(sentence)
    return ' '.join(unique_sentences)

def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()

    return text

def text_normalize(txt):
    processed_text = (re.sub('[^a-zA-Z]', ' ', txt)).lower()
    processed_text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",processed_text)
    processed_text=re.sub("(\\d|\\W)+"," ",processed_text)

    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(processed_text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if not word in stop_words]
    tokens = [i for i in tokens if (tags(i) in ['NN', 'NNP', 'NNS', 'NNPS'])]

    return tokens

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return u" ".join(t.strip() for t in visible_texts)

def readingTime(mytext):
    total_words = len(word_tokenize(mytext))
    estimatedTime = round(total_words/200.0,1)
    return estimatedTime

def connected_component_subgraphs(G):
    for c in nx.connected_components(G):
        yield G.subgraph(c)

def grey_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(0, 0%%, %d%%)" % random.randint(60, 100)

def get_text(url):
    page = urlopen(url)
    soup = BeautifulSoup(page)
    fetched_text = ' '.join(map(lambda p:p.text,soup.find_all('p')))
    return fetched_text

def create_graph(text, common=keys_number):
    tokens = text_normalize(text)
    bigrams=list(nltk.ngrams(tokens, 2))

    bigram_counts = collections.Counter(bigrams)
    bigram_df = pd.DataFrame(bigram_counts.most_common(common), columns=['bigram', 'count'])
    d = bigram_df.set_index('bigram').T.to_dict('records')

    F = nx.Graph()
    for k, v in d[0].items():
            F.add_edge(k[0], k[1], weight=(v*10))
            pos = nx.spring_layout(F, iterations=500)
    return F, pos

def graph_keys(final_text, top_number):
    F,pos = create_graph(final_text.lower())
    nodes = []
    degree = []

    for i in F.nodes():
        nodes.append(i)
        degree.append(F.degree(i))

    x = dict(zip(nodes, degree))
    key_nodes = list({k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)})
    tokens = text_normalize(final_text.lower())

    bigrams=list(nltk.ngrams(tokens, 2))
    bigram_counts = collections.Counter(bigrams)
    bigram_df = pd.DataFrame(bigram_counts.most_common(100), columns=['bigram', 'count'])
    key_bigrams = [' '.join(i) for i in list(bigram_df['bigram'])]

    keys = [i for i in key_bigrams if (((i.split()[0] in key_nodes) or (i.split()[1] in key_nodes)) and (i.split()[0]!=i.split()[1]))][:top_number]
    key_bigrams = ', '.join(keys)

    return key_bigrams

def highlight(txt, keys):
    for k in keys:
        txt = txt.replace(" " + k + " ", " " + "<mark style='background: #A7D1F2; font-size: 105%'><b>" + k+ "</b></mark>" + " ")

    txt = html.unescape(txt)
    return txt

def tags(x):
    return nltk.pos_tag(nltk.word_tokenize(x))[0][1]

def wiki_scrape(topic_name, verbose=True):
    def wiki_link(link):
        try:
            page = wiki_api.page(link)
            if page.exists():
                d = {'page': link, 'text': page.text, 'link': page.fullurl,
                     'categories': list(page.categories.keys())}
                return d
        except:
            return None

    wiki_api = wikipediaapi.Wikipedia(language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI)
    page_name = wiki_api.page(topic_name)
    if not page_name.exists():
        return
    page_links = list(page_name.links.keys())
    progress = tqdm(desc='Links Scraped', unit='', total=len(page_links)) if verbose else None
    sources = [{'page': topic_name, 'text': page_name.text, 'link': page_name.fullurl,
                'categories': list(page_name.categories.keys())}]

    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_link = {executor.submit(wiki_link, link): link for link in page_links}
        for future in concurrent.futures.as_completed(future_link):
            data = future.result()
            progress.update(1) if verbose else None
            if data:
                sources.append(data)
    progress.close() if verbose else None
    blacklist = ('Template', 'Help:', 'Category:', 'Portal:', 'Wikipedia:', 'Talk:')
    sources = pd.DataFrame(sources)
    sources = sources[(len(sources['text']) > 20)
                      & ~(sources['page'].str.startswith(blacklist))]
    sources['categories'] = sources.categories.apply(lambda x: [y[9:] for y in x])
    sources['topic'] = topic_name
    return sources

def wiki_page(page_name):
    wiki_api = wikipediaapi.Wikipedia(language='en', extract_format=wikipediaapi.ExtractFormat.WIKI)
    page_name = wiki_api.page(page_name)
    if not page_name.exists():
        print('page does not exist')
        return
    page_data = {'page': page_name, 'text': page_name.text, 'link': page_name.fullurl,
                 'categories': [[y[9:] for y in list(page_name.categories.keys())]]}
    page_data_df = pd.DataFrame(page_data)
    return page_data_df

def syntax_full(spacy_sentence):
    result=[]
    for token in spacy_sentence:
        if (token.dep == nsubj or token.dep == nsubjpass) and token.head.pos == VERB:
            result.append(token.head)
    if result:
        return True
    else:
        return False

def word_cloud(margin, background_color, tokens):

    stop_words = set(stopwords.words("english"))
    bigrams_list = list(nltk.bigrams(tokens))
    dictionary = [' '.join(tup) for tup in bigrams_list]

    vectorizer = CountVectorizer(ngram_range=(1,1))
    bag_of_words = vectorizer.fit_transform(dictionary)
    words_freq = [(word, bag_of_words.sum(axis=0)[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

    wordcloud = WordCloud(stopwords=stop_words, width=250, height=250, random_state=42, max_words=100, prefer_horizontal=1, margin=margin, background_color=background_color, max_font_size=30)
    wordcloud.generate_from_frequencies(dict(words_freq))
    wordcloud.to_file('images/cloud.png')

    return True

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def filter_triplet(final_text):
    final_text = get_unique_text(final_text)
    doc = nlp(final_text)
    valid_sents=[]

    for s in list(doc.sents):
        if syntax_full(s):
            valid_sents.append(s.text)
    final_text = ' '.join(valid_sents)
    return final_text

def coref_res(rawtext, coref_greedn = 0.5):

    neuralcoref.add_to_pipe(nlp, greedyness = coref_greedn, store_scores=False)
    doc = nlp(rawtext)

    resolved = list(tok.text_with_ws for tok in doc)

    for cluster in doc._.coref_clusters:
        for coref in cluster:
            if coref != cluster.main:
                if coref.text[0].isalpha() and coref.text[0].isupper():

                    main_words_list=word_tokenize(cluster.main.text)
                    main_words_list[0]=main_words_list[0].capitalize()
                    resolved[coref.start] = detokenizer(main_words_list) + doc[coref.end-1].whitespace_

                for i in range(coref.start+1, coref.end):
                    resolved[i] = ""
            else:
                resolved[coref.start] = cluster.main.text + doc[coref.end-1].whitespace_
                for i in range(coref.start+1, coref.end):
                    resolved[i] = ""

    text_resolved = ''.join(resolved)
    nlp.remove_pipe("neuralcoref")

    return text_resolved

def check_min_num_of_clauses(spacy_sentence, n):
    result=[]
    for token in spacy_sentence:
        if (token.dep_ in ['nsubj','nsubjpass','csubj','expl']) and (token.head.pos_ == 'VERB' or token.head.pos_ == 'AUX'):
            result.append(token.head.text)
    if len(result)>=n:
        return True

    else:
        return False

def get_sents_ids_whitelist(spacy_sents):
    whitelist=[]
    i=1
    sents_texts=[]
    for sent in spacy_sents:
        if (sent.text not in sents_texts) and check_min_num_of_clauses(sent,1):
            whitelist.append(i)
            sents_texts.append(sent.text)
        i=i+1
    return(whitelist)

def compress(spacy_sents,sents_whitelist):
    blacklist_tokens=[]
    n=1
    for sent in spacy_sents:
        if (n in sents_whitelist):
            for token in sent:
                if token.dep_ in ['appos','advmod']:
                    token_sub_tree=token.subtree
                    for t in token_sub_tree:
                        blacklist_tokens.append(t.i)

        n=n+1
    return(blacklist_tokens)

def get_list_sents_tokens(spacy_sents,sents_whitelist,blacklist_tokens):
    sents_tokens=[]
    n=1
    for sent in spacy_sents:
        sent_tokens=[]
        if (n in sents_whitelist):
            for token in sent:
                if (token.i not in blacklist_tokens):
                    sent_tokens.append(token.text)
            sents_tokens.append(sent_tokens)
            sent_tokens=[]

        n=n+1
    return(sents_tokens)

def detokenizer(list_of_tokens):
    text_str="".join([" "+w if not w.startswith("'") and not w.startswith("’") and w!='' and w not in string.punctuation else w for w in list_of_tokens]).strip()
    return(text_str)

def sentence_grammar_fix(sentences):
    fixed=[]
    for sent in sentences:

        sent=sent.strip()
        sent=sent.replace('\n','')
        sent=sent.replace('()','')

        sent=re.sub('\s+',' ',sent)
        sent=sent+'.'
        sent=re.sub(r'([,.\-—:])+',r'\1',sent)

        if len(sent)>1:
            if sent[0] in ['.',',','-','—']:
                sent=sent[1:]
        sent=sent.strip()

        if len(sent)>1:
            if sent[0].isalpha():
                sent=sent[0].upper()+sent[1:]
        fixed.append(sent)

    return(fixed)

def spacy_compress(rawtext):

    doc1 = nlp(rawtext)
    sents_whitelist = get_sents_ids_whitelist(doc1.sents)

    tokens_blacklist = compress(doc1.sents,sents_whitelist)
    sents_tokens = get_list_sents_tokens(doc1.sents,sents_whitelist,tokens_blacklist)
    compressed_text_sents = []

    for s in sents_tokens:
        text=detokenizer(s)
        compressed_text_sents.append(text)
    compressed_text_sents=sentence_grammar_fix(compressed_text_sents)
    text =' '.join(compressed_text_sents)

    return(text)

def get_jaccard_sim(a,b):

    a = set(a.split())
    b = set(b.split())
    c = a.intersection(b)

    return float(len(c)/len(a))

def parse_wiki(query):
    wikitext = ''
    df = pd.DataFrame()
    try:
        df = wiki_scrape(query, verbose=False)[:page_number]
        df = df[['text','link','page']]

        wikitext = '; '.join(list(df['text']))
        wiki_keys = list(frozenset(list(df['page'])))
        wiki_keys = ', '.join([i.lower() for i in wiki_keys])
    except:
        wikitext = ''

    return wikitext, wiki_keys, df

def parse_arxiv(query):
    arxivtext = ''
    urls = []
    titles = []

    closest_value = 200
    req = 'https://arxiv.org/search/?query='+query+'&size='+str(closest_value)+'&searchtype=all&source=header&start=0'
    htmlString = get(req)

    soup = BeautifulSoup(htmlString.content, 'html5lib')  
    hrefs = soup.find_all('a', {'href': re.compile(r'arxiv.org/abs/')})

    titles = list(soup.find_all('p', {'class' : 'title is-5 mathjax'}))[:page_number]
    titles_r = [i.text.replace('\n','').replace('  ','') for i in titles]
    titles = ', '.join(titles_r)

    if (len(hrefs) > 0):
        for i in hrefs:
            urls.append(i['href'])

    txt = []
    for i in urls[:page_number]:
        time.sleep(random.randint(1,8))
        soup = BeautifulSoup(get(str(i)).content, 'html5lib')
        
        abstract = ' '.join(soup.find('blockquote').text.replace('\n','').replace('  ',' ').split())       
        abstract = abstract.replace('abstract','').replace('Abstract','')       
        txt.append(abstract)

    arxivtext = re.sub('[^A-Za-z0-9.]+', ' ', '; '.join(txt))
    df = pd.DataFrame(list(zip(txt, urls, titles_r)), columns=['text','link', 'page'])

    return arxivtext, titles, df

# common string #
def longest_common_substring(s1, s2):
  m = [[0] * (1 + len(s2)) for i in range(1 + len(s1))]
  longest, x_longest = 0, 0
  for x in range(1, 1 + len(s1)):
    for y in range(1, 1 + len(s2)):
      if s1[x - 1] == s2[y - 1]:
        m[x][y] = m[x - 1][y - 1] + 1
        if m[x][y] > longest:
          longest = m[x][y]
          x_longest = x
      else:
        m[x][y] = 0
  return s1[x_longest - longest: x_longest]

def longest_common_sentence(s1, s2):
    s1_words = s1.split(' ')
    s2_words = s2.split(' ')
    return ' '.join(longest_common_substring(s1_words, s2_words))

def css(a,b):
    if len(a.split()) > 0:
        score = len(longest_common_sentence(a,b).split())/len(a.split())
    else:    
        score = 0
    return score 
#/common string #

def parse_page(url):
    htmlString = get(url).text
    soup = BeautifulSoup(htmlString, 'html.parser')
    paragraphs = soup.find('body')
    txt = text_from_html(str(paragraphs))

    return txt

def parse_google(query, keys_number, page_number):   
    # load driver
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(chrome_options=chrome_options)
    #/load driver

    # get urls
    google_url = "https://www.google.com/search?q=" + query + "&num=" + str(page_number+1)
    driver.get(google_url)
    time.sleep(randint(1,5))

    soup = BeautifulSoup(driver.page_source,'lxml')
    result_div = soup.find_all('div', attrs={'class': 'g'})

    links = []
    titles = []
    errors = []

    descriptions = []
    for r in result_div:
        try:
            link = r.find('a', href=True)
            title = None
            title = r.find('h3')

            if isinstance(title,Tag):
                title = title.get_text()

            description = None
            description = r.find('span', attrs={'class': 'st'})

            if isinstance(description, Tag):
                description = description.get_text()

            if link != '' and title != '' and description != '':
                links.append(link['href'])
                titles.append(title)
                descriptions.append(description)

        except Exception as e:
            print(e)
            continue

    url_list = links[:(page_number)]
    #/ get urls

    # load content 
    txt = []
    titles = []
    for j in tqdm(url_list):
        delta = random.randint(1,1)
        time.sleep(delta)
        
        try:  
            if str(j).endswith('.pdf'): 
                file_data = parser.from_file(str(j))           
                t = file_data['content']
            else:
                t = parse_page(j)
            txt.append(t)
            titles.append(''.join(sent_tokenize(t)[:3]))
        except:
            print('Parsing error:',str(j))
            errors.append(str(j))
       
    googletext = re.sub('[^A-Za-z0-9.]+', ' ', '; '.join(txt))
    df = pd.DataFrame(list(zip(txt, url_list, titles)), columns=['text','link', 'page'])
   
    return googletext, errors, df

def add_hyperlink(paragraph, text, url):
    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element and a new w:rPr element
    new_run = docx.oxml.shared.OxmlElement('w:r')
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    # Create a new Run object and add the hyperlink into it
    r = paragraph.add_run ()
    r._r.append (hyperlink)

    # A workaround for the lack of a hyperlink style (doesn't go purple after using the link)
    # Delete this if using a template that has the hyperlink style in it
    r.font.color.theme_color = MSO_THEME_COLOR_INDEX.HYPERLINK
    r.font.underline = True

    return hyperlink

def save_doc(final_summary, summary, query, docs_number, readtime):
    sent_list = list(final_summary.split(sep='<hr>'))
    doc = Document()
    style = doc.styles['Normal']
    font = style.font
    font.name = 'Times New Roman'
    font.size = Pt(12)

    hd = doc.add_paragraph()
    hd.alignment = WD_ALIGN_PARAGRAPH.LEFT
    hd.add_run('Summary').bold = True

    if query != 'none':
        hd = doc.add_paragraph('Request: ' + query)

    hd = doc.add_paragraph('Read time: ' + str(readtime) + ' min')
    hd = doc.add_paragraph('Documents: ' + str(docs_number))
    r = hd.add_run()

    for i in sent_list:
        hd.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

        if query != 'none':
            try:
                link = re.search(r"<a href=(.*?)target='_blank'", str(i)).group(1).replace(' ','')
                hd = doc.add_paragraph(striphtml(str(i)).replace('<hr>','').replace('<u>','').replace('More',''))
                add_hyperlink(hd, 'More', link)
            except:
                link = ''
        if query == 'none':
            hd = doc.add_paragraph(striphtml(str(i)).replace('<hr>','').replace('<u>','').replace('More',''))

    doc.save('docs/' + summary + '.docx')
    return True

def get_summary(rawtext, readtime):
    sentences = int(readtime/(np.median([len(i.split()) for i in nltk.sent_tokenize(rawtext)])/200))

    stemmer = Stemmer("english")
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words("english")
    parser = PlaintextParser.from_string(' '.join(sent_tokenize(rawtext)[6:]), Tokenizer("english"))

    text_list = []
    for sentence in summarizer(parser.document, sentences):
        text_list.append(str(sentence))

    txt = ' '.join(sent_tokenize(rawtext)[:6]) + ' '+' '.join(text_list)

    z = 0
    output = []
    pdf_output = []

    for i in nltk.sent_tokenize(txt):
        z = z+1
        output.append('\n\n<hr>' + str(z) + '. ' + str(i))
        pdf_output.append('<hr>' + str(z) + '. ' + str(i))

    txt = ''.join(output) + '<hr>'
    pdf_txt = ''.join(pdf_output) + '<hr>'
    return txt, pdf_txt

Create query:

In [4]:
query = input()

customer development


Parse Wiki:

In [5]:
%%time
wikitext = ''
wikikeys = ''
wiki_time = 0
df_wiki = pd.DataFrame()

if wiki_FT == True:
    start = time.time()
    
    try:
        wikitext = parse_wiki(query)[0]
        wikikeys = parse_wiki(query)[1]
        df_wiki = parse_wiki(query)[2]  
    except:
        print('Parsing error')
    
    end = time.time()
    wiki_time = end - start
    
winsound.Beep(2500, 1000)    

Wall time: 17.1 s


Parse Arxiv:

In [6]:
%%time
arxivtext = ''
arxivkeys = ''
arxiv_time = 0
df_arxiv = pd.DataFrame()
start = time.time()

if arxiv_FT == True:
    try:
        arxivtext = parse_arxiv(query)[0]
        arxivkeys = graph_keys(parse_arxiv(query)[1], keys_number)
        df_arxiv = parse_arxiv(query)[2]
    except:
        print('Parsing error')
    
end = time.time()
arxiv_time = end - start

winsound.Beep(2500, 1000)

Wall time: 1min 19s


Parse Google:

In [7]:
%%time
googletext = ''
googlekeys = ''
google_time = 0
df_google = pd.DataFrame()

if google_FT == True:
    start = time.time()
    
    try:
        gp = parse_google(query, keys_number, page_number)
        googlekeys = graph_keys(gp[0], keys_number)
        googletext = gp[0]   
        df_google = gp[2]
    except:
        print('Parsing error')
    
    end = time.time()
    google_time = end - start
    
winsound.Beep(2500, 1000)    

C:\Users\skamenshchikov\Anaconda3\lib\site-packages\ipykernel_launcher.py:444: DeprecationWarning: use options instead of chrome_options
100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.57s/it]


Wall time: 19.4 s


Concatenate dataframes:

In [8]:
df = df_wiki.append(df_arxiv).append(df_google)

Syntactic filtering:

In [9]:
%%time
start = time.time()

final_text = wikitext + ' ' + arxivtext + ' ' + googletext
final_text = filter_triplet(final_text)
final_text = coref_res(spacy_compress(filter_triplet(final_text)))

end = time.time()
filter_time = end - start

winsound.Beep(2500, 1000)

Wall time: 48.5 s


Get summary:

In [10]:
%%time
start = time.time()
report_summary = get_summary(final_text, readtime)[1]
end = time.time()
summary_time = end - start

Wall time: 3.11 s


Add urls:

In [11]:
pdf_list = []
sent_list = list(report_summary.split(sep='<hr>'))[1:]

for i in sent_list[:-1]:
    try:
        df['score'] = df['text'].apply(lambda x: css(i,x))
        df = df.sort_values(by=['score'], ascending=False)
        if str(df['link'].iloc[0]):
            pdf_list.append(str(str(i) + " <u><a href=" + str(df['link'].iloc[0]) + " target='_blank'>" + "More" + "</a></u>" + "<hr>"))
    except:
        pdf_list.append('')

pdf_summary = ''.join(pdf_list)

Save docx:

In [12]:
save_doc(pdf_summary, 'summary', query, len(df), readtime)

True

Statistics:

In [13]:
total_time = summary_time + filter_time + google_time + arxiv_time + wiki_time
memory_size = int(len(final_text.encode('utf-8'))/1024)
total_time = int(total_time/60)
docs_number = len(df)

print('Precessing time:', total_time, 'min')
print('Memory size:', memory_size, 'mb')
print('Number of documents:', docs_number)

winsound.Beep(2500, 1000)

Precessing time: 2 min
Memory size: 76 mb
Number of documents: 15
